In [1]:
import os
import logging
import time
import psycopg2

import pandas as pd
import numpy as np 
from sqlalchemy import create_engine

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
 

def connect_postgres():
    database = os.environ['POSTGRES_DB']
    user = os.environ['POSTGRES_USER']
    password = os.environ['POSTGRES_PASSWORD']
    host = os.environ['POSTGRES_SERVER']
    port = 5432

    exc, conn, engine = None, None, None

    for _ in range(5):
        try:
            conn = psycopg2.connect(
                database=database, user=user, password=password, host=host, port=port)
        except Exception as e:
            logging.warning("Error connecting to postgres, will retry in 3 sec: %s", e)
            time.sleep(3)
            exc = e
        else:
            logging.info("Connected...")
            logging.info("Everything goes well from Postgres, you're a fu*** pro...")
            
            engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
                user, password, host, port, database
            ))
            break
    else:
        logging.error("Unable to connect to  %s DB", database)
        raise exc
    
    return [conn, engine]


In [2]:
[conn, engine] = connect_postgres()

INFO:root:Connected...
INFO:root:Everything goes well from Postgres, you're a fu*** pro...


In [3]:
df = pd.read_sql("""
    SELECT code, trans_date_id as date, copy.id as books
    FROM public.copytransaction, public.copy, public.student, public.title, public.transactiontype
    WHERE 
        trans_borrower_code = student.id and
        trans_copy_code_id = copy.id and 
        trans_tittle_code_id = title.id and 
        trans_type_id = transactiontype.id and 
        trans_type_code IN ('ISS', 'REN', 'NON', 'PLOAN')
    GROUP BY code, trans_date_id, copy.id
""", con=conn)

/opt/app/.venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
df

,code,date,books
0,198224610,20000928,547476
1,198224610,20000928,547477
2,198224610,20001014,547477
3,198224610,20001117,17499
4,198224610,20001120,172235
...,...,...,...
4360858,201880026,20180305,832497
4360859,201880026,20180307,823259
4360860,201880026,20180313,834960
4360861,201880026,20180320,834962


In [5]:
df = df.astype({col: 'int32' for col in df.select_dtypes('int64').columns})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4360863 entries, 0 to 4360862
Data columns (total 3 columns):
 #   Column  Dtype
---  ------  -----
 0   code    int32
 1   date    int32
 2   books   int32
dtypes: int32(3)
memory usage: 49.9 MB


In [6]:
#exploring the data

# checking null values
df.isnull().sum()

code     0
date     0
books    0
dtype: int64

In [7]:
import numpy as np
import math
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

chunk_max_size = 40000
chunks = int(math.ceil(len(df) / chunk_max_size))

rules = []

In [8]:
for df_chunk in np.array_split(df, chunks):
    # min support is 3 items at least
    min_support = 3/len(df_chunk)
    
    df_chunk.books = df_chunk.books.transform(lambda x: [x])    
    library = df_chunk.groupby(['code','date']).sum()['books'].reset_index(drop=True)
    
    # Hot encode
    encoder = TransactionEncoder()
    transform_library = encoder.fit(library).transform(library)
    transactions = pd.DataFrame(transform_library, columns=encoder.columns_)
    
    # TODO: understand this very well
    frequent_itemsets = fpgrowth(transactions, min_support=min_support, use_colnames=True, max_len=2)
    if not frequent_itemsets.empty:
        rule = association_rules(frequent_itemsets, metric='confidence', min_threshold=0)
        rule['support_transactions_antecedent'] = rule['antecedent support'] * len(df_chunk) # calculate the # of transactions
        rule['support_transactions_consecuente'] = rule['consequent support'] * len(df_chunk) # calculate the # of transactions
        rule['support_transactions'] = rule['support'] * len(df_chunk) # calculate the # of transactions
        
        if not rule.empty:
            rules.append(rule)


In [9]:
result = pd.concat(rules).reset_index(drop=True)
result["antecedents"] = result["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")
result["consequents"] = result["consequents"].apply(lambda x: list(x)[0]).astype("unicode")
result

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,support_transactions_antecedent,support_transactions_consecuente,support_transactions
0,159201,500919,0.000091,0.000136,0.000091,1.000000,7336.333333,0.000091,inf,3.602617,5.403926,3.602617
1,500919,159201,0.000136,0.000091,0.000091,0.666667,7336.333333,0.000091,2.999727,5.403926,3.602617,3.602617
2,42097,178775,0.000136,0.000091,0.000091,0.666667,7336.333333,0.000091,2.999727,5.403926,3.602617,3.602617
3,178775,42097,0.000091,0.000136,0.000091,1.000000,7336.333333,0.000091,inf,3.602617,5.403926,3.602617
4,189961,422662,0.000363,0.000182,0.000091,0.250000,1375.562500,0.000091,1.333091,14.410468,7.205234,3.602617
...,...,...,...,...,...,...,...,...,...,...,...,...
475419,798470,757896,0.000114,0.000189,0.000076,0.666667,3521.600000,0.000076,2.999432,4.502953,7.504922,3.001969
475420,725788,833606,0.000076,0.000454,0.000076,1.000000,2201.000000,0.000076,inf,3.001969,18.011813,3.001969
475421,833606,725788,0.000454,0.000076,0.000076,0.166667,2201.000000,0.000076,1.199909,18.011813,3.001969,3.001969
475422,700173,570318,0.000076,0.000076,0.000076,1.000000,13206.000000,0.000076,inf,3.001969,3.001969,3.001969


In [18]:
result.describe()

/opt/app/.venv/lib/python3.8/site-packages/numpy/lib/function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,antecedent support,consequent support,support,confidence,lift,leverage,conviction,support_transactions_antecedent,support_transactions_consecuente,support_transactions
count,475424.000000,475424.000000,475424.000000,475424.000000,475424.000000,475424.000000,4.754240e+05,475424.000000,475424.000000,475424.000000
mean,0.000287,0.000287,0.000125,0.639412,4221.180687,0.000125,inf,11.375810,11.375810,4.942667
std,0.000322,0.000322,0.000096,0.305394,3560.261704,0.000096,NaN,12.760744,12.760744,3.792906
min,0.000076,0.000076,0.000076,0.009709,6.360167,0.000067,1.008825e+00,3.001969,3.001969,3.001969
25%,0.000120,0.000120,0.000085,0.400000,1382.823529,0.000085,1.666074e+00,4.767387,4.767387,3.372809
50%,0.000178,0.000178,0.000090,0.666667,3071.333333,0.000090,2.999404e+00,7.068242,7.068242,3.569246
75%,0.000325,0.000325,0.000132,1.000000,6068.000000,0.000132,NaN,12.879792,12.879792,5.223779
max,0.008063,0.008063,0.003058,1.000000,13206.000000,0.003048,inf,319.661492,319.661492,121.230101


In [19]:
weak_rules = result[result["confidence"].between(0, 0.4)]
weak_rules.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction,support_transactions_antecedent,support_transactions_consecuente,support_transactions
count,87317.000000,87317.000000,87317.000000,87317.000000,87317.000000,87317.000000,87317.000000,87317.000000,87317.000000,87317.000000
mean,0.000693,0.000418,0.000106,0.187043,916.258012,0.000106,1.239356,27.487401,16.587281,4.207977
std,0.000491,0.000444,0.000050,0.072636,790.545298,0.000049,0.109993,19.479278,17.607125,1.964322
min,0.000265,0.000076,0.000076,0.009709,6.360167,0.000067,1.008825,10.506891,3.001969,3.001969
25%,0.000383,0.000140,0.000085,0.125000,294.387500,0.000084,1.142726,15.173575,5.562249,3.363794
50%,0.000537,0.000264,0.000089,0.200000,682.657143,0.000089,1.248039,21.291085,10.447558,3.522591
75%,0.000830,0.000524,0.000105,0.250000,1320.277778,0.000104,1.333049,32.922353,20.760550,4.163848
max,0.008063,0.008063,0.001022,0.300000,3773.142857,0.001018,1.428418,319.661492,319.661492,40.511596


In [22]:
medium_rules = result[result["confidence"].between(0.41, 0.66)]
medium_rules.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction,support_transactions_antecedent,support_transactions_consecuente,support_transactions
count,75309.000000,75309.000000,75309.000000,75309.000000,75309.000000,75309.000000,75309.000000,75309.000000,75309.000000,75309.000000
mean,0.000273,0.000298,0.000141,0.516207,2760.556483,0.000141,2.091974,10.823628,11.806993,5.595008
std,0.000189,0.000288,0.000101,0.051530,1616.385609,0.000101,0.242120,7.498478,11.435184,4.003005
min,0.000151,0.000076,0.000076,0.410256,62.831461,0.000075,1.692419,6.003938,3.001969,3.001969
25%,0.000175,0.000134,0.000087,0.500000,1472.625000,0.000087,1.999359,6.937008,5.296460,3.468504
50%,0.000210,0.000214,0.000119,0.500000,2409.428571,0.000119,1.999727,8.340837,8.488352,4.721397
75%,0.000305,0.000350,0.000141,0.545455,3760.833333,0.000141,2.199306,12.096068,13.880088,5.601545
max,0.003814,0.007670,0.002133,0.660000,6603.000000,0.002122,2.935623,151.202714,304.057066,84.545939


In [23]:
hard_rules = result[result["confidence"].between(0.67, 1)]
hard_rules.describe()

/opt/app/.venv/lib/python3.8/site-packages/numpy/lib/function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,antecedent support,consequent support,support,confidence,lift,leverage,conviction,support_transactions_antecedent,support_transactions_consecuente,support_transactions
count,193379.000000,193379.000000,193379.000000,193379.000000,193379.000000,193379.000000,1.933790e+05,193379.000000,193379.000000,193379.000000
mean,0.000152,0.000248,0.000140,0.954188,6650.768803,0.000140,inf,6.034563,9.832095,5.537219
std,0.000143,0.000282,0.000122,0.091390,3807.219137,0.000122,NaN,5.664708,11.195554,4.835283
min,0.000076,0.000076,0.000076,0.673077,127.014563,0.000076,3.054171e+00,3.001969,3.001969,3.001969
25%,0.000086,0.000092,0.000086,1.000000,3289.714286,0.000086,NaN,3.402858,3.630569,3.402858
50%,0.000092,0.000153,0.000092,1.000000,5973.250000,0.000092,NaN,3.666081,6.069430,3.666081
75%,0.000174,0.000273,0.000139,1.000000,10919.500000,0.000139,NaN,6.884133,10.839592,5.499122
max,0.003776,0.007873,0.003058,1.000000,13206.000000,0.003048,inf,149.716767,312.121689,121.230101


In [16]:
# TODO: save association rules to a database
result.to_csv('association_rules.csv', index=True, sep='*')